# Assignment 2: The Winter is here
##### This works best with epic battle music. No spoilers present.
<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Tywin Lannister was right when he said: "The great war is between death and life, ice and fire. If we loose, the night will never end"<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;It has been six months since the white walkers' army marched into the north, led by the night king himself on a dead dragon. It has been a battle like never before: never before have men faced such an enemy in battle, never before have men fought so bravely against a united threat, and never before have they been so gravely defeated.<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; While Cersei is in King's landing, brave men have died fighting the great war. Among others, Tyrion is dead, Arya is dead and Jon Snow is dead, again. In a desperate battle, Daenerys leads all her forces in a final stand-off with the dead just south of Winterfell. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Her army defeated, she is now on the run on her dragon in an air battle, being chased by two of her own dragons, the Night king and a dead Jon Snow. Suddenly, the Night king's spear hits Danny's dragon, who, raining blood and fire, falls into ice, taking the lost queen, with him. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Daenerys opens her eyes in a strange place, a place which does not follow the rules of space and time, where the dead souls killed by the dead men are trapped, forever. But who woke her up? There stands near her, Tyrion, with Jorah, Davos, Jon Snow, and everybody else. They all indulge in a heartfelt reunion when someone yells- "But how do we get out?<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Varys sees a talking crystal close by, who asks them of completing a task, which on completion would allow them to go back to the land of the living, with the ultimate tool to defeat the white-walkers and kills the night king, the Dragon-axe. They have summoned you for help, as the task is out of their expertise, to apply a modified CNN to solve the object detection problem on the PASCAL VOC dataset. Varys, the master of whisperers, has used his talents to import the following for you:

In [2]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import os
from PIL import Image


%matplotlib inline
plt.ion()
# You can ask Varys to get you more if you desire

In [3]:
import random
from collections import namedtuple

In [4]:
resnet_input = 224 #size of resnet18 input images

In [5]:
# Cersei chose violence, you choose your hyper-parameters wisely using validation data!
batch_size = 20
num_epochs = 5
learning_rate =  0.001
hyp_momentum = 0.9

## Build the data
The hound who was in charge for getting the data, brought you the following links:
<br/>Training and validation:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
<br/>Testing data:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
<br/>He also told you that the dataset(datascrolls :P) consists of images from of 20 classes, with detection annotations included. The JPEGImages folder houses the images, and the Annotations folder has the object-wise labels for the objects in one xml file per image. You have to extract the object information, ie. the [xmin, ymin] (the top left x,y co-ordinates) and the [xmax, ymax] (the bottom right x,y co-ordinates) of only the objects belonging to the given 20 classes(aeroplane, bicycle, boat, bottle, bus, car, cat, chair, cow, dining table, dog, horse, motorbike, person, potted plant, sheep, train, TV). For parsing the xml file, you can ask Varys to import xml.etree.ElementTree for you. <br/>
<br/> You can then ask Bronn and Jamie to organize the data as follows:
<br/> For every image in the dataset, extract/crop the object patch from the image one by one using their respective co-ordinates:[xmin, ymin, xmax, ymax], resize the image to resnet_input, and store it with its class label information. Do the same for training/validation and test datasets. <br/>
##### Important
You also have to collect data for an extra background class which stands for the class of an object which is not a part of any of the 20 classes. For this, you can crop and resize any random patches from an image. A good idea is to extract patches that have low "intersection over union" with any object present in the image frame from the 20 Pascal VOC classes. The number of background images should be roughly around those of other class objects' images. Hence the total classes turn out to be 21. This is important for applying the sliding window method later.

In [6]:
classes = ('__background__',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor')
counts = [0]*21

In [7]:
Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')

def area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy
    
ra = Rectangle(1, 1, 3, 3)
rb = Rectangle(1, 1, 5, 5)

type(area(ra,rb))

int

In [8]:
def create_train_hierarchical_dirs():
    if not os.path.exists('data/train'):
        os.makedirs('data/train')
        
    for i in range(len(classes)):
        dirname = 'data/train/' + classes[i]
        if not os.path.exists(dirname):
            os.makedirs(dirname)

create_train_hierarchical_dirs()

In [9]:
def jamie_bronn_build_dataset():
    
    dir_names = os.listdir('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations')
    for img_name in dir_names:
        tree = ET.parse('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/' + img_name)
        root = tree.getroot()
        for object1 in root.iter('object'):
            for name1 in object1.iter('name'):
                dirname = name1.text
            for bndbox in object1.iter('bndbox'):
                for xmin in object1.iter('xmin'):
                    x1 = int(float(xmin.text))
                for xmax in object1.iter('xmax'):
                    x2 = int(float(xmax.text))
                for ymin in object1.iter('ymin'):
                    y1 = int(float(ymin.text))
                for ymax in object1.iter('ymax'):
                    y2 = int(float(ymax.text))

                img = Image.open('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages/' + img_name[0:-4] + '.jpg')
                img = img.crop((x1,y1,x2,y2))
                if dirname in classes:
                    img.save('data/train/' + dirname + '/' + str(counts[classes.index(dirname)]) + ".jpg");
                    counts[classes.index(dirname)]+=1
        
jamie_bronn_build_dataset()

In [10]:
def build_background_class():
    dir_names = os.listdir('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations')
    average = int(sum(counts[1:])/len(counts[1:]))
    counts[0] = 0
    while(counts[0]!=average):        
        print(counts[0])
        select = 1
        img_name = random.choice(dir_names)
        tree = ET.parse('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/' + img_name)
        root = tree.getroot()
        img = Image.open('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages/' + img_name[0:-4] + '.jpg')
        imsize = img.size;
        x1 = random.randrange(imsize[0])
        x2 = random.randrange(imsize[0])
        y1 = random.randrange(imsize[1])
        y2 = random.randrange(imsize[1])
        if(x1>x2):
            temp = x1
            x1=x2
            x2=temp
        if(y1>y2):
            temp = y1
            y1=y2
            y2=temp
        print(x1,y1,x2,y2)
        if(x1==x2 or y1==y2):
            continue
        for object1 in root.iter('object'):
            for name1 in object1.iter('name'):
                dirname = name1.text
            for bndbox in object1.iter('bndbox'):
                intersection = 0
                for xmin in object1.iter('xmin'):
                    xa = int(float(xmin.text))
                for xmax in object1.iter('xmax'):
                    xb = int(float(xmax.text))
                for ymin in object1.iter('ymin'):
                    ya = int(float(ymin.text))
                for ymax in object1.iter('ymax'):
                    yb = int(float(ymax.text))
                ra = Rectangle(x1, x2, y1, y2)
                rb = Rectangle(xa, xb, ya, yb)
                if(area(ra,rb)!=None):
                    intersection = area(ra,rb)
                union = ((x2-x1)*(y2-y1))+((xb-xa)*(yb-ya))-intersection
                if(float(intersection)/union>0.5):
                    select = 0

        if(select==1):
            img = img.crop((x1,y1,x2,y2)).resize((resnet_input,resnet_input))
            img.save('data/train/__background__/' + str(counts[0]) + ".jpg");
            counts[0] += 1
    
build_background_class()

0
152 327 274 425
0
187 117 361 251
1
455 218 474 240
2
45 70 106 183
3
310 6 451 335
4
139 102 398 283
5
48 18 100 315
6
358 107 397 243
7
292 141 359 264
8
63 250 113 407
9
125 41 177 143
10
366 92 447 331
11
209 41 340 203
12
375 182 490 291
13
115 67 206 234
14
89 40 477 125
15
21 357 331 373
16
119 173 141 194
16
222 71 451 89
17
139 158 367 302
18
142 139 249 251
19
40 52 177 143
20
0 124 356 217
21
71 137 463 224
22
265 306 311 470
23
59 76 350 424
24
154 11 314 157
25
1 149 208 219
26
82 67 351 371
27
7 336 241 492
28
70 142 267 204
29
172 50 372 367
30
245 246 423 316
31
161 185 448 256
32
150 144 318 219
33
214 29 391 87
34
173 411 180 438
34
230 76 283 314
35
43 360 402 363
36
38 261 272 326
37
123 196 302 332
38
21 40 308 382
39
91 39 363 187
40
225 140 296 365
41
210 154 347 296
42
97 238 121 346
43
43 67 454 241
44
59 104 125 281
45
184 216 361 357
46
278 84 306 289
47
142 214 211 469
47
152 57 426 219
48
324 124 356 241
49
410 22 495 101
50
244 327 298 337
51
142 275 303

429
17 20 289 323
430
27 16 201 315
431
8 265 305 283
432
119 205 482 306
433
109 255 202 310
434
137 26 401 215
435
44 219 202 308
436
179 10 271 69
437
182 273 428 354
438
222 79 483 372
439
327 11 422 57
440
76 245 80 286
441
312 78 462 316
442
129 127 284 172
443
91 108 271 144
444
16 108 240 260
445
175 315 423 319
446
48 161 316 295
447
187 62 425 64
448
15 14 367 33
449
29 256 73 270
450
348 13 364 175
451
134 15 289 342
452
265 210 474 291
453
76 45 348 324
454
25 265 486 360
455
259 68 273 113
456
87 126 95 149
457
197 141 423 170
458
234 265 248 452
459
318 48 494 195
460
86 29 218 335
461
38 161 339 351
462
297 130 352 144
463
21 121 356 244
464
76 76 480 107
465
371 92 487 200
466
155 121 207 225
467
51 185 96 195
468
217 22 304 287
469
101 332 442 345
470
10 104 262 280
471
278 215 295 327
472
340 74 457 302
473
324 210 422 338
474
145 156 327 349
475
218 21 243 151
476
178 178 346 256
477
220 82 271 303
478
102 11 402 283
479
78 77 464 105
480
53 14 221 65
481
139 212 475

In [11]:
class hound_dataset(torch.utils.data.Dataset): # Extend PyTorch's Dataset class
    def __init__(self, root_dir, train, transform=None):
        # Begin
        
        self.root_dir = root_dir
        self.train = train
        self.transform = transform
        if train == True:
            dir1 = '/train/'
        else:
            dir1 = '/test/'
        
        dir_names = os.listdir(root_dir + dir1)
        img_names=[]
        labels = []
        count = 0
        for c in dir_names:
            names = os.listdir(root_dir + dir1 + c)
            N = len(names)
            for n in range(N):
                img_names.append(str(root_dir + dir1 + c + '/' + names[n]))
                labels += [count]
            count+=1
               
        
        self.img_names = img_names
        self.labels = labels  
        
    def __len__(self):
        # Begin
        
        return len(self.img_names)        
        
    def __getitem__(self, idx):
        # Begin
        
        img_pil = Image.open(self.img_names[idx]).convert('RGB')
        return self.transform(img_pil), self.labels[idx]       
        

## Train the network
<br/>You can ask Arya to train the network on the created dataset. This will yield a classification network on the 21 classes of the VOC dataset. 

In [12]:
composed_transform = transforms.Compose([transforms.Scale((resnet_input,resnet_input)),
                                         transforms.ToTensor(),
                                         transforms.RandomHorizontalFlip()])
train_dataset = hound_dataset(root_dir='./data', train=True, transform=composed_transform) # Supply proper root_dir
#test_dataset = hound_dataset(root_dir='./data', train=False, transform=composed_transform) # Supply proper root_dir

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

### Fine-tuning
Litlefinger has brought you a pre-trained network. Fine-tune the network in the following section:

In [13]:
resnet18 = models.resnet18(pretrained=True)

for param in resnet18.parameters():
    param.requires_grad = False

resnet18.fc = nn.Linear(resnet18.fc.in_features, 21)
resnet18.cuda()

# Add code for using CUDA here

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d

In [14]:
criterion = nn.CrossEntropyLoss()
# Update if any errors occur
optimizer = optim.SGD(resnet18.fc.parameters(), learning_rate, hyp_momentum)

In [15]:
def arya_train():
    # Begin
    for epoch in range(num_epochs): 
        resnet18.train()        
        for i, (input1,target) in enumerate(train_loader):
            
            input_var = torch.autograd.Variable(input1).cuda()            
            target = target.cuda(async = True)
            target_var = torch.autograd.Variable(target)
            
            output = resnet18(input_var)            
            loss = criterion(output, target_var)           
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            output = output.float()
            loss = loss.float()
            
            print(str(epoch)+ '---'+str(i) + '--------' +  str(loss) + '-------' +str(output))

In [ ]:
%time arya_train()

# Testing and Accuracy Calculation
Jorah then asks a question, how is this a detection task?<br/>
As everybody wonders, Theon Greyjoy suggests a slding window method to test the above trained trained network on the detection task:<br/>
"We take some windows of varying size and aspect ratios", he mumbled, "and slide it through the test image (considering some stride of pixels) from left to right, and top to bottom, detect the class scores for each of the window, and keep only those which are above a certain threshold value!". "He is right", says Samwell, "I read a similar approach in the paper -Faster RCNN by Ross Girshick in the library, where he uses three diferent scales/sizes and three different aspect ratios, making a total of nine windows per pixel to slide". You need to write the code and use it in testing code to find the predicted boxes and their classes.

In [56]:
def theon_sliding_window():
    # Begin
    boxes=[]
    imres = [[192,192],[192,96],[96,192],[128,128],[128,64],[64,128],[64,64],[64,32],[32,64]]
    for i in range(resnet_input):
        for j in range(resnet_input):
            for window in imres:
                boxes.append([i-window[0]/2, j-window[1]/2, i+window[0]/2, j+window[1]/2])
    return np.asarray(np.array(boxes))


[-96. -48. -96. -64. -32. -64. -32. -16. -32.]


"Wait", says <b>Jon Snow</b>, "The predicted boxes may be too many and we can't deal with all of them. So, I myself will go and apply non_maximum_supression to reduce the number of boxes". You are free to choose the threshold value for non maximum supression, but choose wisely [0,1].

In [62]:
def aegon_targaryen_non_maximum_supression(boxes,threshold = 0.3):
            
        pick = []

        x1 = boxes[:,0]
        y1 = boxes[:,1]
        x2 = boxes[:,2]
        y2 = boxes[:,3]

        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(y2)
        
        while len(idxs) > 0:
            
            last = len(idxs) - 1
            i = idxs[last]
            pick.append(i)
            suppress = [last]
            
            for pos in xrange(0, last):
                
                j = idxs[pos]
               
                xx1 = max(x1[i], x1[j])
                yy1 = max(y1[i], y1[j])
                xx2 = min(x2[i], x2[j])
                yy2 = min(y2[i], y2[j])
               
                w = max(0, xx2 - xx1 + 1)
                h = max(0, yy2 - yy1 + 1)

                overlap = float(w * h) / area[j]

                if overlap > threshold:
                    suppress.append(pos)

            idxs = np.delete(idxs, suppress)

        return boxes[pick]
        
        

In [ ]:
def daenerys_test(resnet18):
    # Write loops for testing the model on the test set
    # Also print out the accuracy of the model

In [ ]:
%time daenerys_test(resnet18)

# Final Showdown
After covering all the steps and passing the accuracy value to the talking crystal, they all pass through to the land of the living, with a wounded Jon Snow armed with the Dragon-axe. After a fierce battle, Jon Snow manages to go face to face with the Night king. Surrounded by battling men and falling bodies, they engage in a ferocious battle, a battle of spear and axe. After a raging fight, Jon manages to sink the axe into the Night king's heart, but not before he gets wounded by the spear. As dead men fall to bones, Daenerys and others rush to his aid, but it is too late. Everyone is in tears as they look towards the man of honour, Jon Snow, lying in Daenerys's arms when he says his last words: "The night has ended. Winter is finally over!"